# Visualizar el CSV 

In [33]:
import pandas as pd

# Cargar el CSV
csv_path = 'data/export.csv'
df = pd.read_csv(csv_path, encoding='utf-16', sep='\t')

# 1. Ver columnas del CSV
print(f"📊 El CSV tiene {len(df.columns)} columnas.")
print("📝 Columnas del CSV:", list(df.columns))

# 2. Mostrar los primeros registros para ver el formato
df.head()


📊 El CSV tiene 74 columnas.
📝 Columnas del CSV: ['sequence_id ', 'razon_social                                                                                        ', 'nombre_comercial                                                                     ', 'sector                                           ', 'actividad_2                                                                                                                                                                                                                                                                                                             ', 'actividad                                                                                                                                                                                                                                                                                                               ', 'cif            ', 'direccion_del_negocio                

/var/folders/09/3xcrkv750gd3cy0wyd9255rm0000gn/T/ipykernel_11978/1629525645.py:5: DtypeWarning: Columns (69) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding='utf-16', sep='\t')


,sequence_id,razon_social,nombre_comercial,sector,actividad_2,actividad,cif,direccion_del_negocio,codigo_postal,localidad,...,facturacion_ordenar,resultado_2023,url_edicion_impresa,capital_social,username,fee_id,terms_and_conditions_acceptance_date,expires_on,created_date,modified_date
0,1,Iteisa Desarrollo y Sistemas SL ...,...,Informática/Nuevas tecnologías,Desarrollo de aplicaciones informáticas ...,Desarrollo de aplicaciones informáticas ...,,Benidorm 8 Bajo ...,39005,,...,,...,,3100,Datos,2,,,"febrero 12, 2025 5:59 pm","febrero 12, 2025 5:59 pm"
1,2,KLM Aotearoa SL ...,...,Servicios a la empresa,Asesoramiento y consultoría en el campo económ...,Asesoramiento y consultoría en el campo económ...,,Rentería Reyes 9 Bajo ...,39740,,...,,...,,3000,Datos,2,,,"febrero 12, 2025 5:20 pm","febrero 12, 2025 5:44 pm"
2,3,Peña Miera SRL ...,...,Agricultura/Ganadería,"Actividad agrícola, ganadera y forestal. Comer...","Actividad agrícola, ganadera y forestal. Comer...",,San Fernando s/n ...,39409,,...,,...,,320000,Datos,2,,,"febrero 12, 2025 5:17 pm","febrero 12, 2025 5:17 pm"
3,4,APJ Wind SL ...,...,Banca/Servicios financieros/Seguros,Participación en cualquier tipo de sociedades ...,Participación en cualquier tipo de sociedades ...,,Barrio Rubo s/n ...,39478,Boo,...,,...,,3100,Datos,,,,"febrero 12, 2025 5:15 pm","febrero 12, 2025 5:15 pm"
4,5,Gilda y Asociados SL ...,...,Banca/Servicios financieros/Seguros,Actividades auxiliares a los servicios financi...,Actividades auxiliares a los servicios financi...,,Camino Calero-Rionegro 400 ...,39795,,...,,...,,3000,Datos,2,,,"febrero 12, 2025 5:13 pm","febrero 12, 2025 5:13 pm"


## Limpieza del CSV 
### Formatos fechas, decimales, 

In [44]:
import pandas as pd
from datetime import datetime

# 📌 1️⃣ Cargar el CSV
csv_path = "data/export.csv"
df = pd.read_csv(csv_path, encoding="utf-16", sep="\t", low_memory=False, dtype=str)

# 📌 2️⃣ Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# 📌 3️⃣ Corregir nombres de columnas mal formateados
if "director__gerente" in df.columns:
    df.rename(columns={"director__gerente": "director_gerente"}, inplace=True)

# 📌 4️⃣ Convertir Fechas Generales
def convertir_fecha(fecha):
    """Convierte fechas completas al formato MySQL (YYYY-MM-DD HH:MM:SS)"""
    if fecha and isinstance(fecha, str):
        fecha = fecha.strip()
        try:
            meses_espanol = {
                "enero": "January", "febrero": "February", "marzo": "March",
                "abril": "April", "mayo": "May", "junio": "June",
                "julio": "July", "agosto": "August", "septiembre": "September",
                "octubre": "October", "noviembre": "November", "diciembre": "December"
            }
            for mes_es, mes_en in meses_espanol.items():
                if mes_es in fecha.lower():
                    fecha = fecha.lower().replace(mes_es, mes_en)
                    break
            return datetime.strptime(fecha, "%B %d, %Y %I:%M %p").strftime("%Y-%m-%d %H:%M:%S")
        except ValueError:
            return None  
    return None  

# Aplicar conversión de fechas generales
columnas_fecha = ["created_date", "modified_date"]
for col in columnas_fecha:
    if col in df.columns:
        df[col] = df[col].apply(convertir_fecha)

# 📌 5️⃣ Convertir Fechas de `fecha_de_fundacion_de_la_empresa`
def convertir_fecha_fundacion(fecha):
    """Convierte fechas en la columna 'fecha_de_fundacion_de_la_empresa' a formato MySQL (YYYY-MM-DD)"""
    if fecha and isinstance(fecha, str):
        fecha = fecha.strip()
        try:
            if fecha.isdigit() and len(fecha) == 4:
                return f"{fecha}-01-01"
            if fecha.isdigit() and len(fecha) == 2:
                return f"20{fecha}-01-01"
            meses_abreviados = {"ene": "01", "feb": "02", "mar": "03", "abr": "04", "may": "05", "jun": "06", "jul": "07", "ago": "08", "sep": "09", "oct": "10", "nov": "11", "dic": "12"}
            if "-" in fecha:
                mes, anio = fecha.split("-")
                mes = mes.lower()
                if mes in meses_abreviados and anio.isdigit():
                    return f"20{anio}-{meses_abreviados[mes]}-01"
        except ValueError:
            return None  
    return None  

if "fecha_de_fundacion_de_la_empresa" in df.columns:
    df["fecha_de_fundacion_de_la_empresa"] = df["fecha_de_fundacion_de_la_empresa"].apply(convertir_fecha_fundacion)

# 📌 6️⃣ Convertir Números y Decimales
def limpiar_decimal(valor):
    if valor is None or pd.isna(valor) or valor == "":
        return None
    try:
        valor = valor.replace(",", ".")
        valor = "".join(c for c in valor if c.isdigit() or c == ".")
        return round(float(valor), 2)
    except ValueError:
        return None

columnas_decimales = [
    "facturacion_2004", "facturacion_2005", "facturacion_2006", "facturacion_2007",
    "facturacion_2008", "facturacion_2009", "facturacion_2010", "facturacion_2011",
    "facturacion_2012", "facturacion_2013", "facturacion_2014", "facturacion_2015",
    "facturacion_2016", "facturacion_2017", "resultado_2017", "facturacion_2018",
    "resultado_2018", "facturacion_2019", "resultado_2019", "facturacion_2020",
    "resultado_2020", "facturacion_2021", "resultado_2021", "director_gerente",
    "ver_facturaciones_anteriores_a_2018", "resultado_2024", "facturacion_2024",
    "resultado_2022", "facturacion_2023", "facturacion_2022", "facturacion_ordenar",
    "resultado_2023"
]

for col in columnas_decimales:
    if col in df.columns:
        df[col] = df[col].apply(limpiar_decimal)

# 📌 7️⃣ Convertir `NaN` en `None` para que MySQL lo reconozca como `NULL`
df = df.where(pd.notnull(df), None)

# 📌 8️⃣ Asegurar codificación UTF-8 en los textos
df = df.applymap(lambda x: x.encode('utf-8', 'ignore').decode('utf-8') if isinstance(x, str) else x)

# 📌 9️⃣ Guardar el nuevo CSV corregido
nuevo_csv_path = "data/export_limpio.csv"
df.to_csv(nuevo_csv_path, encoding="utf-8", sep="\t", index=False, na_rep="")  # Evita que 'nan' se guarde

print(f"✅ Archivo export_limpio.csv listo para la BBDD")



/var/folders/09/3xcrkv750gd3cy0wyd9255rm0000gn/T/ipykernel_11978/1768602736.py:96: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.encode('utf-8', 'ignore').decode('utf-8') if isinstance(x, str) else x)


✅ Archivo export_limpio.csv listo para la BBDD


In [45]:
csv_path = "data/export_limpio.csv"
df_limpio = pd.read_csv(csv_path, encoding="utf-8", sep="\t", low_memory=False, dtype=str)
df_limpio.head(70)

,sequence_id,razon_social,nombre_comercial,sector,actividad_2,actividad,cif,direccion_del_negocio,codigo_postal,localidad,...,facturacion_ordenar,resultado_2023,url_edicion_impresa,capital_social,username,fee_id,terms_and_conditions_acceptance_date,expires_on,created_date,modified_date
0,1,Iteisa Desarrollo y Sistemas SL ...,...,Informática/Nuevas tecnologías,Desarrollo de aplicaciones informáticas ...,Desarrollo de aplicaciones informáticas ...,,Benidorm 8 Bajo ...,39005,,...,NaN,NaN,,3100,Datos,2,,,2025-02-12 17:59:00,2025-02-12 17:59:00
1,2,KLM Aotearoa SL ...,...,Servicios a la empresa,Asesoramiento y consultoría en el campo económ...,Asesoramiento y consultoría en el campo económ...,,Rentería Reyes 9 Bajo ...,39740,,...,NaN,NaN,,3000,Datos,2,,,2025-02-12 17:20:00,2025-02-12 17:44:00
2,3,Peña Miera SRL ...,...,Agricultura/Ganadería,"Actividad agrícola, ganadera y forestal. Comer...","Actividad agrícola, ganadera y forestal. Comer...",,San Fernando s/n ...,39409,,...,NaN,NaN,,320000,Datos,2,,,2025-02-12 17:17:00,2025-02-12 17:17:00
3,4,APJ Wind SL ...,...,Banca/Servicios financieros/Seguros,Participación en cualquier tipo de sociedades ...,Participación en cualquier tipo de sociedades ...,,Barrio Rubo s/n ...,39478,Boo,...,NaN,NaN,,3100,Datos,,,,2025-02-12 17:15:00,2025-02-12 17:15:00
4,5,Gilda y Asociados SL ...,...,Banca/Servicios financieros/Seguros,Actividades auxiliares a los servicios financi...,Actividades auxiliares a los servicios financi...,,Camino Calero-Rionegro 400 ...,39795,,...,NaN,NaN,,3000,Datos,2,,,2025-02-12 17:13:00,2025-02-12 17:13:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,66,Bioestética Rosa Blu SL ...,...,Servicios a las personas,Peluquería y tratamientos de belleza ...,Peluquería y tratamientos de belleza ...,,"Los Infantes, 95 ...",39005,,...,NaN,NaN,,32.000,cant.negocios,2,,,2024-12-02 17:22:00,2024-12-02 17:22:00
66,67,Mascamadillo Consultoría y Proyectos SL ...,...,"Construcción, servicios inmobiliarios y afines","Gestión, dirección y ejecución de proyectos de...","Gestión, dirección y ejecución de proyectos de...",,Parque Empresarial Besaya nave 44 ...,39538,,...,NaN,NaN,,3.000,cant.negocios,2,,,2024-12-02 17:22:00,2024-12-02 17:22:00
67,68,Mabe Learning Trips SL ...,...,Educación,Enseñanza fuera de establecimientos permanente...,Enseñanza fuera de establecimientos permanente...,,El Parque 35 Bajo ...,39108,,...,NaN,NaN,,3.000,cant.negocios,2,,,2024-12-02 17:22:00,2024-12-02 17:22:00
68,69,Proyectos y Reformas de Cantabria SL ...,...,"Construcción, servicios inmobiliarios y afines",Construcción especializada. Reformas ...,Construcción especializada. Reformas ...,,La Hoz 246 ...,39694,,...,NaN,NaN,,3.000,cant.negocios,2,,,2024-12-02 17:22:00,2024-12-02 17:22:00


creacion BBDD

In [46]:
import pandas as pd
import mysql.connector

# 📌 1️⃣ Conectar a MySQL y Crear la Base de Datos
conexion = mysql.connector.connect(
    host='localhost',
    user='root',
    password=''
)
cursor = conexion.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS directorio;")
cursor.close()
conexion.close()

print("✅ Base de datos 'directorio' creada o ya existente.")


✅ Base de datos 'directorio' creada o ya existente.


creacion de la tabla 

In [47]:

# 📌 2️⃣ Conectar nuevamente y crear la tabla
conexion = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='directorio'
)
cursor = conexion.cursor()

tabla_sql = """
CREATE TABLE IF NOT EXISTS directorioEmpresasCantabria (
    sequence_id INT AUTO_INCREMENT PRIMARY KEY,
    razon_social VARCHAR(255),
    nombre_comercial VARCHAR(255),
    sector VARCHAR(255),
    actividad_2 VARCHAR(255),
    actividad VARCHAR(255),
    cif VARCHAR(50),
    direccion_del_negocio TEXT,
    codigo_postal VARCHAR(20),
    localidad VARCHAR(255),
    municipio VARCHAR(255),
    sitio_web_del_negocio VARCHAR(255),
    fax_del_negocio VARCHAR(50),
    numero_telefonico_del_negocio VARCHAR(50),
    descripcion_corta_del_negocio TEXT,
    fecha_de_fundacion_de_la_empresa DATETIME,
    correo_electronico_de_contacto_del_negocio VARCHAR(255),
    poligono_industrial VARCHAR(255),
    facturacion_2004 FLOAT,
    facturacion_2005 FLOAT,
    facturacion_2006 FLOAT,
    facturacion_2007 FLOAT,
    facturacion_2008 FLOAT,
    facturacion_2009 FLOAT,
    facturacion_2010 FLOAT,
    facturacion_2011 FLOAT,
    facturacion_2012 FLOAT,
    facturacion_2013 FLOAT,
    facturacion_2014 FLOAT,
    facturacion_2015 FLOAT,
    facturacion_2016 FLOAT,
    facturacion_2017 FLOAT,
    resultado_2017 FLOAT,
    facturacion_2018 FLOAT,
    resultado_2018 FLOAT,
    facturacion_2019 FLOAT,
    resultado_2019 FLOAT,
    facturacion_2020 FLOAT,
    resultado_2020 FLOAT,
    facturacion_2021 FLOAT,
    resultado_2021 FLOAT,
    director_gerente VARCHAR(255),
    cargo_1 VARCHAR(255),
    cargo_2 VARCHAR(255),
    cargo_3 VARCHAR(255),
    cargo_4 VARCHAR(255),
    cargo_5 VARCHAR(255),
    cargo_6 VARCHAR(255),
    cargo_7 VARCHAR(255),
    cargo_8 VARCHAR(255),
    cargo_9 VARCHAR(255),
    numero_de_trabajadores INT,
    exporta VARCHAR(10),
    que_productos_exporta TEXT,
    estado_de_la_sociedad VARCHAR(255),
    sector_ordenar VARCHAR(255),
    no_trabajadores_ordenar INT,
    ver_facturaciones_anteriores_a_2018 VARCHAR(10),
    municipio_ordenar VARCHAR(255),
    resultado_2024 FLOAT,
    facturacion_2024 FLOAT,
    resultado_2022 FLOAT,
    facturacion_2023 FLOAT,
    facturacion_2022 FLOAT,
    facturacion_ordenar FLOAT,
    resultado_2023 FLOAT,
    url_edicion_impresa VARCHAR(255),
    capital_social FLOAT,
    username VARCHAR(255),
    fee_id INT,
    terms_and_conditions_acceptance_date DATETIME,
    expires_on DATETIME,
    created_date DATETIME,
    modified_date DATETIME
);
"""

cursor.execute(tabla_sql)
conexion.commit()

print("✅ Tabla 'directorioEmpresasCantabria' creada o ya existente.")



✅ Tabla 'directorioEmpresasCantabria' creada o ya existente.


In [ ]:
print("Columnas del CSV:", df_limpio.columns.tolist())


Columnas del CSV: ['sequence_id', 'razon_social', 'nombre_comercial', 'sector', 'actividad_2', 'actividad', 'cif', 'direccion_del_negocio', 'codigo_postal', 'localidad', 'municipio', 'sitio_web_del_negocio', 'fax_del_negocio', 'numero_telefonico_del_negocio', 'descripcion_corta_del_negocio', 'fecha_de_fundacion_de_la_empresa', 'correo_electronico_de_contacto_del_negocio', 'poligono_industrial', 'facturacion_2004', 'facturacion_2005', 'facturacion_2006', 'facturacion_2007', 'facturacion_2008', 'facturacion_2009', 'facturacion_2010', 'facturacion_2011', 'facturacion_2012', 'facturacion_2013', 'facturacion_2014', 'facturacion_2015', 'facturacion_2016', 'facturacion_2017', 'resultado_2017', 'facturacion_2018', 'resultado_2018', 'facturacion_2019', 'resultado_2019', 'facturacion_2020', 'resultado_2020', 'facturacion_2021', 'resultado_2021', 'director_gerente', 'cargo_1', 'cargo_2', 'cargo_3', 'cargo_4', 'cargo_5', 'cargo_6', 'cargo_7', 'cargo_8', 'cargo_9', 'numero_de_trabajadores', 'export

volcado de datos 

In [ ]:
import pandas as pd
import mysql.connector

# 📌 1️⃣ Conectar a MySQL
conexion = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='directorio'
)
cursor = conexion.cursor()

df_limpio = pd.read_csv(csv_path, encoding="utf-8", sep="\t", low_memory=False, dtype=str)

# 📌 5️⃣ Obtener las columnas de la tabla en MySQL
cursor.execute("SHOW COLUMNS FROM directorioEmpresasCantabria")
columnas_mysql = [col[0] for col in cursor.fetchall()]

# 📌 6️⃣ Asegurar que las columnas del CSV coincidan con MySQL
columnas_csv = df_limpio.columns.tolist()
columnas_faltantes = [col for col in columnas_csv if col not in columnas_mysql]

if columnas_faltantes:
    print("⚠️ ERROR: Estas columnas en el CSV NO existen en MySQL:", columnas_faltantes)
    raise ValueError("Corrige los nombres de las columnas antes de continuar.")

# 📌 7️⃣ Crear la consulta de inserción dinámica con las columnas correctas
columnas_insert = ", ".join(columnas_csv)
placeholders = ", ".join(["%s"] * len(columnas_csv))
sql_insert = f"INSERT INTO directorioEmpresasCantabria ({columnas_insert}) VALUES ({placeholders})"

# 📌 8️⃣ Insertar los datos fila por fila
for _, fila in df_limpio.iterrows():
    valores = tuple(fila)
    cursor.execute(sql_insert, valores)

# 📌 9️⃣ Confirmar cambios en MySQL
conexion.commit()

# 📌 🔟 Cerrar la conexión
cursor.close()
conexion.close()

print("✅ ¡Datos importados correctamente en MySQL!")


ProgrammingError: 1054 (42S22): Unknown column 'nan' in 'field list'